In [1]:
# TO DO
#
# MVP: create a version that allows us to pass and no more

Copied from assignment 2. NEEDS WORK!!!

In [2]:
# Organize data for pytorch DataLoader
import os
import shutil
import random
import torch

#DATA_DIR = 'D:/Kurssit/Data Science/Deep learning/dl2021-image-corpus-proj/'
DATA_DIR = '/Users/hartih/Documents/School/Deep learning/Final_project/dl2021-image-corpus-proj/'
ANNOTATIONS_DIR = DATA_DIR + 'annotations/'
IMAGES_DIR = DATA_DIR + 'images/'

# New fodlers for train, test, and dev sets
TRAIN_DIR = DATA_DIR + 'train/'
DEV_DIR = DATA_DIR + 'dev/'
TEST_DIR = DATA_DIR + 'test/'

annotations = ["baby",
               "bird",
               "car",
               "clouds",
               "dog",
               "female",
               "flower",
               "male",
               "night",
               "people",
               "portrait",
               "river",
               "sea",
               "tree"]

In [3]:
# Create labels for images
image_file_names = os.listdir(IMAGES_DIR)
dict_labels = {}
for image_file_name in image_file_names:  # Initiate label tensors
    if os.path.isfile(IMAGES_DIR + image_file_name):
        dict_labels[image_file_name] = torch.zeros(14)
for i in range(len(annotations)):  # Fill label tensors with 1's if found in one of the annotations text files
    with open(ANNOTATIONS_DIR + annotations[i] + ".txt") as f:
        for row in f:
            row = "im" + row.strip() + ".jpg"
            dict_labels[row][i] = 1

In [4]:
dict_labels

{'im1976.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im12710.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im14361.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im17668.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.]),
 'im10107.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'im11219.jpg': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im19645.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im16576.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'im19123.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.]),
 'im16210.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'im10661.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0.]),
 'im3807.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im8952.jpg': ten

In [5]:
# Split the data to train, test, and dev
os.makedirs(TRAIN_DIR)
os.makedirs(TEST_DIR)
os.makedirs(DEV_DIR)
for image_file_name in image_file_names:
    if os.path.isfile(IMAGES_DIR + image_file_name):
        division = random.randint(1, 4)
        if division == 1 or division == 2:
            shutil.copyfile(IMAGES_DIR + image_file_name, TRAIN_DIR + image_file_name)
        if division == 3:
            shutil.copyfile(IMAGES_DIR + image_file_name, DEV_DIR + image_file_name)
        if division == 4:
            shutil.copyfile(IMAGES_DIR + image_file_name, TEST_DIR + image_file_name)

In [6]:
print( len(os.listdir(TEST_DIR)) + len(os.listdir(TRAIN_DIR)) + len( os.listdir(DEV_DIR)) == len(os.listdir(IMAGES_DIR)) )
print(len(os.listdir(TEST_DIR)))
print(len(os.listdir(TRAIN_DIR)))
print(len(os.listdir(DEV_DIR)))
print(len(os.listdir(IMAGES_DIR)))


True
5193
9798
5009
20000


In [8]:
import torch.utils.data
from PIL import Image

# Enable creating train, test, and dev test datasets for PyTorch
class myDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir=IMAGES_DIR, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.images = [root_dir + img for img in os.listdir(root_dir)]                
      
    def __len__(self):
        return len(self.images)       

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = img = Image.open(img_path)     
       
        if self.transform:
            img = self.transform(img)     

        return img, dict_labels[img_path.split("/")[-1]]

In [10]:
from torchvision import transforms
import numpy as np

#Calculate mean and std with whole dataset
imageNet = True  # If imageNet is True, then we use the values from imagenet, else we calculate them from the dataset

if imageNet:
  mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]  # Imagenet mean and std
else:
  def mean_std(loader):
    images, labels = next(iter(loader))
    # shape of images = [b,c,w,h]
    mean, std = images.mean([0,2,3]), images.std([0,2,3])
    return mean, std

  mean_transform = transforms.Compose([
                                          transforms.Grayscale(num_output_channels=3),
                                          transforms.Resize(256),                    
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor()])

  #Splitting these was the easiest due to memory allocation problem, you could only used IMAGES_DIR if this is not a problem, note the equal divide
  train_mean_set = myDataset(TRAIN_DIR, transform=mean_transform)
  train_data_loader = torch.utils.data.DataLoader(dataset=train_mean_set, batch_size=len(train_mean_set), shuffle=False, num_workers=0)
  dev_mean_set = myDataset(DEV_DIR, transform=mean_transform)
  dev_data_loader = torch.utils.data.DataLoader(dataset=dev_mean_set, batch_size=len(dev_mean_set), shuffle=False, num_workers=0)
  test_mean_set = myDataset(TEST_DIR, transform=mean_transform)
  test_data_loader = torch.utils.data.DataLoader(dataset=test_mean_set, batch_size=len(test_mean_set), shuffle=False, num_workers=0)

  mean_train, std_train = mean_std(train_data_loader)
  mean_dev, std_dev = mean_std(dev_data_loader)
  mean_test, std_test = mean_std(test_data_loader)
  mean = list((np.array(mean_train)+np.array(mean_dev)+np.array(mean_test))/3)
  std = list((np.array(std_train)+np.array(std_dev)+np.array(std_test))/3)
print("Mean:", mean)
print("std:", std)

#Imagenet mean and std
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
                                        #transforms.RandomHorizontalFlip(),
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.RandomCrop(128),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomRotation(2),
                                        transforms.ToTensor()])
test_transform = transforms.Compose([
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.ToTensor()])

transform = transforms.Compose([            #[1]
 transforms.Grayscale(num_output_channels=3),
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=mean,                #[6]
 std=std                 #[7]
 )]
)

# Create datasets for CNN
train_set = myDataset(TRAIN_DIR, transform=transform)
test_set = myDataset(TEST_DIR, transform=transform)
dev_set = myDataset(DEV_DIR, transform=transform)

Mean: [0.485, 0.456, 0.406]
std: [0.229, 0.224, 0.225]


In [11]:
import torch 

# Calculate the amount of correctly predicted as well as total predictions done
def calc_correct(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)  # Since our neural network does not apply sigmoid
    correct_dict = {'tot': [0,0]}  # First number in value is correct ones, the second one is total amount
    correct_dict['tot_strict'] = [0,0] # All correct
    for i in range(len(pred)): # [100,14]
        all_correct = 0
        total = 0
        # estim_pred_array = []
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            # estim_pred_array.append(estim_pred)
            if annotations[j] not in correct_dict.keys():
                correct_dict[annotations[j]] = [0,0]
            correct_dict['tot'][1] += 1
            correct_dict[annotations[j]][1] += 1
            correct_dict['tot'][0] += int(estim_pred == target[i][j])
            correct_dict[annotations[j]][0] += int(estim_pred == target[i][j])
            all_correct += int(estim_pred == target[i][j])
            total += 1
        correct_dict['tot_strict'][1] += 1
        correct_dict['tot_strict'][0] += int(total==all_correct)
    # To-do how many pictures were entirely correct (accuracy)
    return correct_dict


In [12]:
def class_evaluation(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)
    
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    negative = 0
    positive = 0
    
    for i in range(len(pred)): # [100,14]
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                negative += 1
                if estim_pred == 1:
                    false_positive += 1
                if estim_pred == 0:
                    true_negative += 1
            
            # Positive target values
            if target[i][j] == 1:
                positive += 1
                if estim_pred == 1:
                    true_positive += 1
                if estim_pred == 0:
                    false_negative += 1
    
    result = {"true_positive": true_positive,
            "false_positive": false_positive,
            "true_negative": true_negative,
            "false_negative": false_negative,
            "negative": negative,
            "positive": positive}
                    
    return result

In [13]:
def class_evaluation_by_annotation(pred: torch.Tensor, target: torch.Tensor):
    
    pred = torch.sigmoid(pred)
    
    # Initiate vslues for every annotation
    eval_dict = {}
    for a in range(len(annotations)):
        eval_dict[annotations[a]] = {"true_positive": 0,
                                        "false_positive": 0,
                                        "true_negative": 0,
                                        "false_negative": 0,
                                        "negative": 0,
                                        "positive": 0}
    
    for i in range(len(pred)):
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                eval_dict[annotations[j]]["negative"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["false_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["true_negative"] += 1
            
            # Positive target values
            if target[i][j] == 1:
                eval_dict[annotations[j]]["positive"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["true_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["false_negative"] += 1
                    
    return eval_dict

In [14]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets, models
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math


#--- hyperparameters ---
N_EPOCHS = 15
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 100
LR = 0.001
WEIGHT_DECAY = 0.00
MOMENTUM = 0.1


#--- fixed constants ---
NUM_CLASSES = 14
NUM_CHANNELS = 3



# --- Dataset initialization ---

# We transform image files' contents to tensors
# Plus, we can add random transformations to the training data if we like
# Think on what kind of transformations may be meaningful for this data.
# Eg., horizontal-flip is definitely a bad idea for sign language data.
# You can use another transformation here if you find a better one.


# Create Pytorch data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)
dev_loader = torch.utils.data.DataLoader(dataset=dev_set, shuffle=False)

alexnet = models.alexnet(pretrained=True)
resnet = models.resnet101(pretrained=True)

alexnet.train()
resnet.train()

#--- model ---
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        
        self.alex = alexnet
        for p in self.alex.parameters():
            p.requires_grad=False
        
        self.resnet = resnet
        for p in self.resnet.parameters():
            p.requires_grad=False
        
        self.lin = nn.Sequential(
            nn.Linear(2000, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 250),
            nn.ReLU(),
            nn.Linear(250, num_classes)
        )

    def forward(self, x):
        
        out_alex = self.alex(x)
        out_resnet = self.resnet(x)
        
        combined_pretrained_output = torch.cat((out_alex, out_resnet), dim=1)
        
        out = self.lin(combined_pretrained_output)
        
        return out



#--- set up ---
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
loss_function = nn.BCEWithLogitsLoss()

dev_loss = math.inf
dev_losses = []
dev_accuracies = []
stop_early = False

#--- training ---
for epoch in range(N_EPOCHS):
    if stop_early:
        break
    train_loss = 0
    train_correct = {annotation: [0,0] for annotation in annotations}
    train_correct['tot'] = [0,0]
    train_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}
    total = 0

    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += len(data)
        train_loss += loss.item()
        new_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_correct[annotation]
            train_correct[annotation][0] += new[0]
            train_correct[annotation][1] += new[1]
        train_correct['tot'][0] += new_correct['tot'][0]
        train_correct['tot'][1] += new_correct['tot'][1]
        train_correct['tot_strict'][0] += new_correct['tot_strict'][0]
        train_correct['tot_strict'][1] += new_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]

        print("------------------------")
        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (epoch+1, batch_num+1, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct['tot'][0] / train_correct['tot'][1], train_correct['tot'][0], train_correct['tot'][1],
               100. * train_correct['tot_strict'][0] / train_correct['tot_strict'][1], train_correct['tot_strict'][0], train_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

    cur_dev_loss = 0
    dev_correct = {annotation: [0,0] for annotation in annotations}
    dev_correct['tot'] = [0,0]
    dev_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

    with torch.no_grad():
        for batch_num, (data, target) in enumerate(dev_loader):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            loss = loss_function(pred, target)

            cur_dev_loss += loss.item()
            new_dev_correct = calc_correct(pred, target)
            for annotation in annotations:
                new = new_dev_correct[annotation]
                dev_correct[annotation][0] += new[0]
                dev_correct[annotation][1] += new[1]
            dev_correct['tot'][0] += new_dev_correct['tot'][0]
            dev_correct['tot'][1] += new_dev_correct['tot'][1]
            dev_correct['tot_strict'][0] += new_dev_correct['tot_strict'][0]
            dev_correct['tot_strict'][1] += new_dev_correct['tot_strict'][1]
            
            evaluations = class_evaluation(pred, target)
            evaluation["true_positive"] += evaluations["true_positive"]
            evaluation["false_positive"] += evaluations["false_positive"]
            evaluation["true_negative"] += evaluations["true_negative"]
            evaluation["false_negative"] += evaluations["false_negative"]
            evaluation["positive"] += evaluations["positive"]
            evaluation["negative"] += evaluations["negative"]

        current_loss = cur_dev_loss / (len(dev_loader) + 1)
        dev_losses.append(current_loss)
        current_accuracy = {annotation: 100. * dev_correct[annotation][0] / dev_correct[annotation][1]  for annotation in annotations}  # Accuracies for all classes 
        current_accuracy['tot'] = 100. * dev_correct['tot'][0] / dev_correct['tot'][1]
        current_accuracy['tot_strict'] = 100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1]
        dev_accuracies.append(current_accuracy)

        if current_loss <= dev_loss:
            dev_loss = current_loss
        # else:
        #     stop_early = True

        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Dev Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
            (batch_num+1, len(dev_loader), cur_dev_loss / (len(dev_loader) + 1), 
            100. * dev_correct['tot'][0] / dev_correct['tot'][1], dev_correct['tot'][0], dev_correct['tot'][1],
            100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1], dev_correct['tot_strict'][0], dev_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")


/Users/hartih/opt/anaconda3/envs/dl2/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hartih/opt/anaconda3/envs/dl2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /Users/hartih/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|████████████████████████████████████████| 233M/233M [00:57<00:00, 4.28MB/s]
/Users/hartih/opt/anaconda3/envs/dl2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: Us

------------------------
Training: Epoch 1 - Batch 1/98: Loss: 0.6851 | Train Acc: 55.357% (775/1400) | Strict Acc: 0.000% (0/100)
True positive rate: 31.731% (33/104)
False negative rate: 68.269% (71/104)
True negative rate: 57.253% (742/1296)
False positive rate: 42.747% (554/1296)
------------------------
------------------------
Training: Epoch 1 - Batch 2/98: Loss: 0.5484 | Train Acc: 72.143% (2020/2800) | Strict Acc: 16.500% (33/200)
True positive rate: 13.415% (33/246)
False negative rate: 86.585% (213/246)
True negative rate: 77.800% (1987/2554)
False positive rate: 22.200% (567/2554)
------------------------
------------------------
Training: Epoch 1 - Batch 3/98: Loss: 0.4540 | Train Acc: 79.238% (3328/4200) | Strict Acc: 23.667% (71/300)
True positive rate: 16.760% (60/358)
False negative rate: 83.240% (298/358)
True negative rate: 85.060% (3268/3842)
False positive rate: 14.940% (574/3842)
------------------------
------------------------
Training: Epoch 1 - Batch 4/98: Los

------------------------
Training: Epoch 1 - Batch 27/98: Loss: 0.2008 | Train Acc: 92.529% (34976/37800) | Strict Acc: 45.407% (1226/2700)
True positive rate: 36.486% (999/2738)
False negative rate: 63.514% (1739/2738)
True negative rate: 96.905% (33977/35062)
False positive rate: 3.095% (1085/35062)
------------------------
------------------------
Training: Epoch 1 - Batch 28/98: Loss: 0.1984 | Train Acc: 92.615% (36305/39200) | Strict Acc: 45.679% (1279/2800)
True positive rate: 37.267% (1061/2847)
False negative rate: 62.733% (1786/2847)
True negative rate: 96.949% (35244/36353)
False positive rate: 3.051% (1109/36353)
------------------------
------------------------
Training: Epoch 1 - Batch 29/98: Loss: 0.1962 | Train Acc: 92.665% (37622/40600) | Strict Acc: 45.586% (1322/2900)
True positive rate: 37.636% (1111/2952)
False negative rate: 62.364% (1841/2952)
True negative rate: 96.980% (36511/37648)
False positive rate: 3.020% (1137/37648)
------------------------
--------------

------------------------
Training: Epoch 1 - Batch 52/98: Loss: 0.1657 | Train Acc: 93.804% (68289/72800) | Strict Acc: 50.038% (2602/5200)
True positive rate: 44.323% (2358/5320)
False negative rate: 55.677% (2962/5320)
True negative rate: 97.705% (65931/67480)
False positive rate: 2.295% (1549/67480)
------------------------
------------------------
Training: Epoch 1 - Batch 53/98: Loss: 0.1645 | Train Acc: 93.841% (69630/74200) | Strict Acc: 50.226% (2662/5300)
True positive rate: 44.639% (2423/5428)
False negative rate: 55.361% (3005/5428)
True negative rate: 97.724% (67207/68772)
False positive rate: 2.276% (1565/68772)
------------------------
------------------------
Training: Epoch 1 - Batch 54/98: Loss: 0.1634 | Train Acc: 93.878% (70972/75600) | Strict Acc: 50.352% (2719/5400)
True positive rate: 44.867% (2478/5523)
False negative rate: 55.133% (3045/5523)
True negative rate: 97.741% (68494/70077)
False positive rate: 2.259% (1583/70077)
------------------------
-------------

------------------------
Training: Epoch 1 - Batch 77/98: Loss: 0.1481 | Train Acc: 94.422% (101787/107800) | Strict Acc: 52.896% (4073/7700)
True positive rate: 48.602% (3754/7724)
False negative rate: 51.398% (3970/7724)
True negative rate: 97.959% (98033/100076)
False positive rate: 2.041% (2043/100076)
------------------------
------------------------
Training: Epoch 1 - Batch 78/98: Loss: 0.1477 | Train Acc: 94.441% (103130/109200) | Strict Acc: 53.026% (4136/7800)
True positive rate: 48.591% (3794/7808)
False negative rate: 51.409% (4014/7808)
True negative rate: 97.972% (99336/101392)
False positive rate: 2.028% (2056/101392)
------------------------
------------------------
Training: Epoch 1 - Batch 79/98: Loss: 0.1472 | Train Acc: 94.467% (104480/110600) | Strict Acc: 53.190% (4202/7900)
True positive rate: 48.568% (3833/7892)
False negative rate: 51.432% (4059/7892)
True negative rate: 97.993% (100647/102708)
False positive rate: 2.007% (2061/102708)
------------------------


...

------------------------
Training: Epoch 14 - Batch 83/98: Loss: 0.0413 | Train Acc: 98.357% (114291/116200) | Strict Acc: 81.699% (6781/8300)
True positive rate: 86.759% (7260/8368)
False negative rate: 13.241% (1108/8368)
True negative rate: 99.257% (107031/107832)
False positive rate: 0.743% (801/107832)
------------------------
------------------------
Training: Epoch 14 - Batch 84/98: Loss: 0.0413 | Train Acc: 98.355% (115666/117600) | Strict Acc: 81.679% (6861/8400)
True positive rate: 86.748% (7338/8459)
False negative rate: 13.252% (1121/8459)
True negative rate: 99.255% (108328/109141)
False positive rate: 0.745% (813/109141)
------------------------
------------------------
Training: Epoch 14 - Batch 85/98: Loss: 0.0413 | Train Acc: 98.354% (117041/119000) | Strict Acc: 81.624% (6938/8500)
True positive rate: 86.716% (7422/8559)
False negative rate: 13.284% (1137/8559)
True negative rate: 99.256% (109619/110441)
False positive rate: 0.744% (822/110441)
-----------------------

------------------------
Training: Epoch 15 - Batch 10/98: Loss: 0.0379 | Train Acc: 98.443% (13782/14000) | Strict Acc: 83.500% (835/1000)
True positive rate: 87.174% (870/998)
False negative rate: 12.826% (128/998)
True negative rate: 99.308% (12912/13002)
False positive rate: 0.692% (90/13002)
------------------------
------------------------
Training: Epoch 15 - Batch 11/98: Loss: 0.0398 | Train Acc: 98.403% (15154/15400) | Strict Acc: 83.000% (913/1100)
True positive rate: 87.580% (952/1087)
False negative rate: 12.420% (135/1087)
True negative rate: 99.224% (14202/14313)
False positive rate: 0.776% (111/14313)
------------------------
------------------------
Training: Epoch 15 - Batch 12/98: Loss: 0.0405 | Train Acc: 98.387% (16529/16800) | Strict Acc: 82.833% (994/1200)
True positive rate: 88.384% (1050/1188)
False negative rate: 11.616% (138/1188)
True negative rate: 99.148% (15479/15612)
False positive rate: 0.852% (133/15612)
------------------------
------------------------

------------------------
Training: Epoch 15 - Batch 36/98: Loss: 0.0397 | Train Acc: 98.397% (49592/50400) | Strict Acc: 82.444% (2968/3600)
True positive rate: 87.761% (3191/3636)
False negative rate: 12.239% (445/3636)
True negative rate: 99.224% (46401/46764)
False positive rate: 0.776% (363/46764)
------------------------
------------------------
Training: Epoch 15 - Batch 37/98: Loss: 0.0398 | Train Acc: 98.396% (50969/51800) | Strict Acc: 82.432% (3050/3700)
True positive rate: 87.573% (3284/3750)
False negative rate: 12.427% (466/3750)
True negative rate: 99.240% (47685/48050)
False positive rate: 0.760% (365/48050)
------------------------
------------------------
Training: Epoch 15 - Batch 38/98: Loss: 0.0395 | Train Acc: 98.408% (52353/53200) | Strict Acc: 82.579% (3138/3800)
True positive rate: 87.588% (3366/3843)
False negative rate: 12.412% (477/3843)
True negative rate: 99.250% (48987/49357)
False positive rate: 0.750% (370/49357)
------------------------
----------------

------------------------
Training: Epoch 15 - Batch 61/98: Loss: 0.0393 | Train Acc: 98.411% (84043/85400) | Strict Acc: 82.557% (5036/6100)
True positive rate: 87.282% (5367/6149)
False negative rate: 12.718% (782/6149)
True negative rate: 99.274% (78676/79251)
False positive rate: 0.726% (575/79251)
------------------------
------------------------
Training: Epoch 15 - Batch 62/98: Loss: 0.0391 | Train Acc: 98.419% (85428/86800) | Strict Acc: 82.645% (5124/6200)
True positive rate: 87.346% (5460/6251)
False negative rate: 12.654% (791/6251)
True negative rate: 99.279% (79968/80549)
False positive rate: 0.721% (581/80549)
------------------------
------------------------
Training: Epoch 15 - Batch 63/98: Loss: 0.0391 | Train Acc: 98.418% (86805/88200) | Strict Acc: 82.651% (5207/6300)
True positive rate: 87.313% (5547/6353)
False negative rate: 12.687% (806/6353)
True negative rate: 99.280% (81258/81847)
False positive rate: 0.720% (589/81847)
------------------------
----------------

------------------------
Training: Epoch 15 - Batch 86/98: Loss: 0.0386 | Train Acc: 98.434% (118515/120400) | Strict Acc: 82.767% (7118/8600)
True positive rate: 87.621% (7538/8603)
False negative rate: 12.379% (1065/8603)
True negative rate: 99.267% (110977/111797)
False positive rate: 0.733% (820/111797)
------------------------
------------------------
Training: Epoch 15 - Batch 87/98: Loss: 0.0385 | Train Acc: 98.438% (119898/121800) | Strict Acc: 82.793% (7203/8700)
True positive rate: 87.660% (7622/8695)
False negative rate: 12.340% (1073/8695)
True negative rate: 99.267% (112276/113105)
False positive rate: 0.733% (829/113105)
------------------------
------------------------
Training: Epoch 15 - Batch 88/98: Loss: 0.0384 | Train Acc: 98.438% (121276/123200) | Strict Acc: 82.807% (7287/8800)
True positive rate: 87.639% (7714/8802)
False negative rate: 12.361% (1088/8802)
True negative rate: 99.269% (113562/114398)
False positive rate: 0.731% (836/114398)
-----------------------

In [15]:
#--- test ---

alexnet.eval()
resnet.eval()

test_loss = 0
test_correct = {annotation: [0,0] for annotation in annotations}
test_correct['tot'] = [0,0]
test_correct['tot_strict'] = [0,0]
evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

evaluation_by_annotation = {}
for a in range(len(annotations)):
    evaluation_by_annotation[annotations[a]] = {"true_positive": 0,
                                                "false_positive": 0,
                                                "true_negative": 0,
                                                "false_negative": 0,
                                                "negative": 0,
                                                "positive": 0}

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        test_loss += loss.item()
        new_test_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_test_correct[annotation]
            test_correct[annotation][0] += new[0]
            test_correct[annotation][1] += new[1]
        test_correct['tot'][0] += new_test_correct['tot'][0]
        test_correct['tot'][1] += new_test_correct['tot'][1]
        test_correct['tot_strict'][0] += new_test_correct['tot_strict'][0]
        test_correct['tot_strict'][1] += new_test_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]
        
        evaluations_by_annotation = class_evaluation_by_annotation(pred, target)
        print(evaluation_by_annotation)
        
        for a in range(len(annotations)):
            evaluation_by_annotation[annotations[a]]["true_positive"] += evaluations_by_annotation[annotations[a]]["true_positive"]
            evaluation_by_annotation[annotations[a]]["false_positive"] += evaluations_by_annotation[annotations[a]]["false_positive"]
            evaluation_by_annotation[annotations[a]]["true_negative"] += evaluations_by_annotation[annotations[a]]["true_negative"]
            evaluation_by_annotation[annotations[a]]["false_negative"] += evaluations_by_annotation[annotations[a]]["false_negative"]
            evaluation_by_annotation[annotations[a]]["positive"] += evaluations_by_annotation[annotations[a]]["positive"]
            evaluation_by_annotation[annotations[a]]["negative"] += evaluations_by_annotation[annotations[a]]["negative"]
        
        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (batch_num+1, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct['tot'][0] / test_correct['tot'][1], test_correct['tot'][0], test_correct['tot'][1],
               100. * test_correct['tot_strict'][0] / test_correct['tot_strict'][1], test_correct['tot_strict'][0], test_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")
        
print(dev_losses)
print(dev_accuracies)

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'male': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'night': {'true_pos

{'baby': {'true_positive': 0, 'false_positive': 1, 'true_negative': 396, 'false_negative': 3, 'negative': 397, 'positive': 3}, 'bird': {'true_positive': 2, 'false_positive': 0, 'true_negative': 396, 'false_negative': 2, 'negative': 396, 'positive': 4}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 395, 'false_negative': 5, 'negative': 395, 'positive': 5}, 'clouds': {'true_positive': 3, 'false_positive': 1, 'true_negative': 376, 'false_negative': 20, 'negative': 377, 'positive': 23}, 'dog': {'true_positive': 1, 'false_positive': 0, 'true_negative': 396, 'false_negative': 3, 'negative': 396, 'positive': 4}, 'female': {'true_positive': 19, 'false_positive': 18, 'true_negative': 309, 'false_negative': 54, 'negative': 327, 'positive': 73}, 'flower': {'true_positive': 5, 'false_positive': 1, 'true_negative': 378, 'false_negative': 16, 'negative': 379, 'positive': 21}, 'male': {'true_positive': 21, 'false_positive': 20, 'true_negative': 326, 'false_negative': 33, 'negative

{'baby': {'true_positive': 0, 'false_positive': 1, 'true_negative': 795, 'false_negative': 4, 'negative': 796, 'positive': 4}, 'bird': {'true_positive': 5, 'false_positive': 0, 'true_negative': 790, 'false_negative': 5, 'negative': 790, 'positive': 10}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 783, 'false_negative': 17, 'negative': 783, 'positive': 17}, 'clouds': {'true_positive': 6, 'false_positive': 3, 'true_negative': 746, 'false_negative': 45, 'negative': 749, 'positive': 51}, 'dog': {'true_positive': 6, 'false_positive': 1, 'true_negative': 785, 'false_negative': 8, 'negative': 786, 'positive': 14}, 'female': {'true_positive': 53, 'false_positive': 34, 'true_negative': 634, 'false_negative': 79, 'negative': 668, 'positive': 132}, 'flower': {'true_positive': 7, 'false_positive': 2, 'true_negative': 763, 'false_negative': 28, 'negative': 765, 'positive': 35}, 'male': {'true_positive': 39, 'false_positive': 35, 'true_negative': 654, 'false_negative': 72, 'neg

{'baby': {'true_positive': 0, 'false_positive': 1, 'true_negative': 1192, 'false_negative': 7, 'negative': 1193, 'positive': 7}, 'bird': {'true_positive': 7, 'false_positive': 0, 'true_negative': 1182, 'false_negative': 11, 'negative': 1182, 'positive': 18}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 1180, 'false_negative': 20, 'negative': 1180, 'positive': 20}, 'clouds': {'true_positive': 8, 'false_positive': 5, 'true_negative': 1123, 'false_negative': 64, 'negative': 1128, 'positive': 72}, 'dog': {'true_positive': 8, 'false_positive': 1, 'true_negative': 1179, 'false_negative': 12, 'negative': 1180, 'positive': 20}, 'female': {'true_positive': 73, 'false_positive': 49, 'true_negative': 967, 'false_negative': 111, 'negative': 1016, 'positive': 184}, 'flower': {'true_positive': 11, 'false_positive': 3, 'true_negative': 1147, 'false_negative': 39, 'negative': 1150, 'positive': 50}, 'male': {'true_positive': 59, 'false_positive': 49, 'true_negative': 992, 'false_ne

{'baby': {'true_positive': 2, 'false_positive': 2, 'true_negative': 1589, 'false_negative': 7, 'negative': 1591, 'positive': 9}, 'bird': {'true_positive': 9, 'false_positive': 0, 'true_negative': 1570, 'false_negative': 21, 'negative': 1570, 'positive': 30}, 'car': {'true_positive': 1, 'false_positive': 0, 'true_negative': 1578, 'false_negative': 21, 'negative': 1578, 'positive': 22}, 'clouds': {'true_positive': 9, 'false_positive': 8, 'true_negative': 1489, 'false_negative': 94, 'negative': 1497, 'positive': 103}, 'dog': {'true_positive': 11, 'false_positive': 3, 'true_negative': 1569, 'false_negative': 17, 'negative': 1572, 'positive': 28}, 'female': {'true_positive': 95, 'false_positive': 66, 'true_negative': 1293, 'false_negative': 146, 'negative': 1359, 'positive': 241}, 'flower': {'true_positive': 15, 'false_positive': 4, 'true_negative': 1530, 'false_negative': 51, 'negative': 1534, 'positive': 66}, 'male': {'true_positive': 88, 'false_positive': 65, 'true_negative': 1315, 'fals

...

{'baby': {'true_positive': 2, 'false_positive': 2, 'true_negative': 3183, 'false_negative': 13, 'negative': 3185, 'positive': 15}, 'bird': {'true_positive': 15, 'false_positive': 0, 'true_negative': 3140, 'false_negative': 45, 'negative': 3140, 'positive': 60}, 'car': {'true_positive': 3, 'false_positive': 0, 'true_negative': 3145, 'false_negative': 52, 'negative': 3145, 'positive': 55}, 'clouds': {'true_positive': 29, 'false_positive': 18, 'true_negative': 2995, 'false_negative': 158, 'negative': 3013, 'positive': 187}, 'dog': {'true_positive': 25, 'false_positive': 4, 'true_negative': 3132, 'false_negative': 39, 'negative': 3136, 'positive': 64}, 'female': {'true_positive': 185, 'false_positive': 130, 'true_negative': 2584, 'false_negative': 301, 'negative': 2714, 'positive': 486}, 'flower': {'true_positive': 32, 'false_positive': 5, 'true_negative': 3072, 'false_negative': 91, 'negative': 3077, 'positive': 123}, 'male': {'true_positive': 161, 'false_positive': 125, 'true_negative': 

{'baby': {'true_positive': 2, 'false_positive': 2, 'true_negative': 3581, 'false_negative': 15, 'negative': 3583, 'positive': 17}, 'bird': {'true_positive': 18, 'false_positive': 0, 'true_negative': 3527, 'false_negative': 55, 'negative': 3527, 'positive': 73}, 'car': {'true_positive': 3, 'false_positive': 0, 'true_negative': 3540, 'false_negative': 57, 'negative': 3540, 'positive': 60}, 'clouds': {'true_positive': 34, 'false_positive': 20, 'true_negative': 3373, 'false_negative': 173, 'negative': 3393, 'positive': 207}, 'dog': {'true_positive': 30, 'false_positive': 5, 'true_negative': 3522, 'false_negative': 43, 'negative': 3527, 'positive': 73}, 'female': {'true_positive': 220, 'false_positive': 145, 'true_negative': 2890, 'false_negative': 345, 'negative': 3035, 'positive': 565}, 'flower': {'true_positive': 38, 'false_positive': 5, 'true_negative': 3461, 'false_negative': 96, 'negative': 3466, 'positive': 134}, 'male': {'true_positive': 187, 'false_positive': 146, 'true_negative': 

{'baby': {'true_positive': 2, 'false_positive': 2, 'true_negative': 3977, 'false_negative': 19, 'negative': 3979, 'positive': 21}, 'bird': {'true_positive': 19, 'false_positive': 0, 'true_negative': 3919, 'false_negative': 62, 'negative': 3919, 'positive': 81}, 'car': {'true_positive': 4, 'false_positive': 0, 'true_negative': 3934, 'false_negative': 62, 'negative': 3934, 'positive': 66}, 'clouds': {'true_positive': 35, 'false_positive': 21, 'true_negative': 3765, 'false_negative': 179, 'negative': 3786, 'positive': 214}, 'dog': {'true_positive': 34, 'false_positive': 5, 'true_negative': 3914, 'false_negative': 47, 'negative': 3919, 'positive': 81}, 'female': {'true_positive': 248, 'false_positive': 172, 'true_negative': 3201, 'false_negative': 379, 'negative': 3373, 'positive': 627}, 'flower': {'true_positive': 43, 'false_positive': 6, 'true_negative': 3841, 'false_negative': 110, 'negative': 3847, 'positive': 153}, 'male': {'true_positive': 208, 'false_positive': 156, 'true_negative':

{'baby': {'true_positive': 2, 'false_positive': 2, 'true_negative': 4377, 'false_negative': 19, 'negative': 4379, 'positive': 21}, 'bird': {'true_positive': 19, 'false_positive': 0, 'true_negative': 4316, 'false_negative': 65, 'negative': 4316, 'positive': 84}, 'car': {'true_positive': 5, 'false_positive': 0, 'true_negative': 4328, 'false_negative': 67, 'negative': 4328, 'positive': 72}, 'clouds': {'true_positive': 38, 'false_positive': 23, 'true_negative': 4141, 'false_negative': 198, 'negative': 4164, 'positive': 236}, 'dog': {'true_positive': 37, 'false_positive': 6, 'true_negative': 4301, 'false_negative': 56, 'negative': 4307, 'positive': 93}, 'female': {'true_positive': 276, 'false_positive': 192, 'true_negative': 3518, 'false_negative': 414, 'negative': 3710, 'positive': 690}, 'flower': {'true_positive': 47, 'false_positive': 6, 'true_negative': 4230, 'false_negative': 117, 'negative': 4236, 'positive': 164}, 'male': {'true_positive': 223, 'false_positive': 175, 'true_negative':

{'baby': {'true_positive': 2, 'false_positive': 2, 'true_negative': 4777, 'false_negative': 19, 'negative': 4779, 'positive': 21}, 'bird': {'true_positive': 20, 'false_positive': 0, 'true_negative': 4708, 'false_negative': 72, 'negative': 4708, 'positive': 92}, 'car': {'true_positive': 7, 'false_positive': 0, 'true_negative': 4715, 'false_negative': 78, 'negative': 4715, 'positive': 85}, 'clouds': {'true_positive': 41, 'false_positive': 24, 'true_negative': 4522, 'false_negative': 213, 'negative': 4546, 'positive': 254}, 'dog': {'true_positive': 40, 'false_positive': 6, 'true_negative': 4695, 'false_negative': 59, 'negative': 4701, 'positive': 99}, 'female': {'true_positive': 300, 'false_positive': 206, 'true_negative': 3841, 'false_negative': 453, 'negative': 4047, 'positive': 753}, 'flower': {'true_positive': 47, 'false_positive': 6, 'true_negative': 4618, 'false_negative': 129, 'negative': 4624, 'positive': 176}, 'male': {'true_positive': 243, 'false_positive': 186, 'true_negative':

In [16]:
for i in evaluation_by_annotation:
    print("------------------------")
    print(i + ":")
    print("")
    print("positive: " + str(evaluation_by_annotation[i]["positive"]))
    print("true_positive: " + str(evaluation_by_annotation[i]["true_positive"]))
    print("false_negative: " + str(evaluation_by_annotation[i]["false_negative"]))
    print("")
    print("negative: " + str(evaluation_by_annotation[i]["negative"]))
    print("true_negative: " + str(evaluation_by_annotation[i]["true_negative"]))
    print("false_positive: " + str(evaluation_by_annotation[i]["false_positive"]))
        


------------------------
baby:

positive: 23
true_positive: 3
false_negative: 20

negative: 5170
true_negative: 5168
false_positive: 2
------------------------
bird:

positive: 100
true_positive: 22
false_negative: 78

negative: 5093
true_negative: 5093
false_positive: 0
------------------------
car:

positive: 91
true_positive: 8
false_negative: 83

negative: 5102
true_negative: 5102
false_positive: 0
------------------------
clouds:

positive: 272
true_positive: 41
false_negative: 231

negative: 4921
true_negative: 4897
false_positive: 24
------------------------
dog:

positive: 110
true_positive: 43
false_negative: 67

negative: 5083
true_negative: 5077
false_positive: 6
------------------------
female:

positive: 810
true_positive: 327
false_negative: 483

negative: 4383
true_negative: 4157
false_positive: 226
------------------------
flower:

positive: 194
true_positive: 49
false_negative: 145

negative: 4999
true_negative: 4992
false_positive: 7
------------------------
male:

po